# Date Mining Assignment

In [1]:
import numpy as np
import pandas as pd
from IPython.display import display, Markdown, Latex
pd.set_option('max_colwidth', 1000)

import re, os
for d in ["src","data"]: os.makedirs(d, exist_ok=True)
    
DEBUG=False

## Load Dataset

In [2]:
URL = "https://kmurphy.bitbucket.io/modules/Data_Mining_2/topics/08-Text_Mining/20-Mining_Dates/files/"

for filename, dest in [("public.csv", "src")]:
    
    source = f"{URL}/{filename}"
    target = f"{dest}/{filename}"

    if not os.path.isfile(target):
        print (f"Downloading remote file {filename} to folder {dest}", sep="")
        import urllib.request
        urllib.request.urlretrieve(source, target)
    else:
        print(f"Using local copy of {filename} in folder {dest}")

Using local copy of public.csv in folder src


In [3]:
df = pd.read_csv("src/public.csv")
df.head()

,Code,Raw
0,1012720972,".12, Noember 16- bad reaction to SpiceK2 - synthetic MJ- admitted to Crete Manor, Mcalester."
1,1039370009,".April, 5 97: made a phone call to Mom and Mom commented that he was talking very fast, hard to interrupt, but was in super happy spirits, so didn't make a big deal of it."
2,1039574613,"A pleasant 28 yo woman with no formal psychiatric history and with a h/o SCCA of the right tongue (s/p partial glossectomy and neck dissection in 8/1974) referred to psycho-oncology for assistance with adjustment issues following recovery. The patient does not meet criteria for a major mood or anxiety disorder. She is not at imminent risk of harm to self or others. She would benefit from psychotherapy to help her integrate her experience of cancer and the break-up of her engagement, and to think through how to continue to create a life for herself moving forward."
3,1039963589,"October 7, 01 [report_end]"
4,1048901075,"July, 4, 01 Primary Care Doctor:"


## Outline of Solution Pipeline

 * First perform a cleaning step. This greatly reduces the number of regular expressions needed.
     - you to figure out specifics here
   
 * Define two helper functions to simply application of regular expression and to display which parts of dataset are matched / not matched.
 * Build a sequence of regular expressions (starting from most restrictive to avoid false positives) to apply to dataset.
 * At each match/iteration use examples of unmatched rows to determine next possible regular expression.

---

## Cleaning Step

As a result of this step, the dataframe will have columns

 * __Code__ unchanged
 * __Raw__ unchanged
 * __Data__ cleaned version of __Raw__
 * __Iter__ number of regex that matched this row (for development purposes)
 * __Match__ regex object result for this row (for development purposes, you might not use)
 * __Day__ day (or zero if not set)
 * __Month__ month (or zero if not set)
 * __Year__ year (or zero if not set)

### 1. Removing uneeded characters

In [4]:
df['Data'] = df["Raw"].replace(
    {'\(':'',
     '\)':'',
     '\.':' ',
     '\,':' ',
     '-':' ',
     '\:':'',
     '\+':'',
     '\~':'',
     '\s+':' '
    }, regex=True)

df['Data'] = df["Data"].replace('\s+', ' ', regex=True)
# df['Data'] = df["Data"].apply(lambda x: [word.zfill(2) for word in x.split() if len(word) == 1 & word.isnumeric()])

df["Iter"] = 0        
df["Match"] = 0     
df["Day"] = 0
df["Month"] = 0
df["Year"] = 0


### 2. Convert to lower

In [5]:
for line in df['Data']:
    df['Data'] = df['Data'].replace(line, line.lower())

### 3. Fixing month typos and convert to abbreviated

In [6]:
from fuzzywuzzy import fuzz

months = {
    "january": "jan",
    "february": "feb", 
    "march": "mar", 
    "april": "apr", 
    "june": "jun", 
    "july": "jul", 
    "august": "aug", 
    "september": "sep", 
    "october": "oct", 
    "november": "nov", 
    "december": "dec"
}

for index, line in df['Data'].items():
    for word in line.split():
        for key, val in months.items():
            ratio = fuzz.ratio(word, key)
            if ratio > 90:
                df['Data'] = df['Data'].replace(line, line.replace(word, val))
                if DEBUG:
                    print("Match between: {0} - {1}, {2}%".format(word, key, ratio))

c:\users\luke\appdata\local\programs\python\python36\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


### 4. Adding Leading Zeroes

In [7]:
for index, line in df['Data'].items():
    words = line.split()
    indices = []
    
    for word in words:
        
        # if single digit, get index and apply zfill (index used to avoid applying zfill to other occurances)
        if word.isnumeric() and len(word) == 1:
            indices.append(words.index(word))
            
        for idx in indices:
            words[idx] = words[idx].zfill(2)
        
        # if single digit exists within forward slash
        if len(word.split('/')) > 1:
            for i in word.split('/'):
                if i.isnumeric() and len(i) == 1:
                    formatted = '/'.join([item.zfill(2) for item in (word.split('/'))])
                    df['Data'] = df['Data'].replace(line, line.replace(word, formatted))
                    if DEBUG:
                        print ("Converting {0} to {1}".format(word, formatted))
                        
    df['Data'] = df['Data'].replace(line, ' '.join(words))

### 5. Convert Month Abbreviations

In [8]:
# months = {
#     "jan": "january", 
#     "feb": "february", 
#     "mar": "march", 
#     "apr": "april", 
#     "jun": "june", 
#     "jul": "july", 
#     "aug": "august", 
#     "sep": "september",
#     "oct": "october",
#     "nov": "november", 
#     "dec": "december"
# }

# for line in df['Data']:
#     for word in line.split():
#           for key, val in months.items(): 
#             if word == key:
#                 df['Data'] = df['Data'].replace(line, line.replace(word, val))

### 6. Sample of cleaned data 

In [9]:
df.head(10)
# df.to_csv("cleaned_only.csv", index=False)

,Code,Raw,Data,Iter,Match,Day,Month,Year
0,1012720972,".12, Noember 16- bad reaction to SpiceK2 - synthetic MJ- admitted to Crete Manor, Mcalester.",12 nov 16 bad reaction to spicek2 synthetic mj admitted to crete manor mcalester,0,0,0,0,0
1,1039370009,".April, 5 97: made a phone call to Mom and Mom commented that he was talking very fast, hard to interrupt, but was in super happy spirits, so didn't make a big deal of it.",apr 05 97 made a phone call to mom and mom commented that he was talking very fast hard to interrupt but was in super happy spirits so didn't make a big deal of it,0,0,0,0,0
2,1039574613,"A pleasant 28 yo woman with no formal psychiatric history and with a h/o SCCA of the right tongue (s/p partial glossectomy and neck dissection in 8/1974) referred to psycho-oncology for assistance with adjustment issues following recovery. The patient does not meet criteria for a major mood or anxiety disorder. She is not at imminent risk of harm to self or others. She would benefit from psychotherapy to help her integrate her experience of cancer and the break-up of her engagement, and to think through how to continue to create a life for herself moving forward.",a pleasant 28 yo woman with no formal psychiatric history and with a h/o scca of the right tongue s/p partial glossectomy and neck dissection in 08/1974 referred to psycho oncology for assistance with adjustment issues following recovery the patient does not meet criteria for a major mood or anxiety disorder she is not at imminent risk of harm to self or others she would benefit from psychotherapy to help her integrate her experience of cancer and the break up of her engagement and to think through how to continue to create a life for herself moving forward,0,0,0,0,0
3,1039963589,"October 7, 01 [report_end]",oct 07 01 [report_end],0,0,0,0,0
4,1048901075,"July, 4, 01 Primary Care Doctor:",jul 04 01 primary care doctor,0,0,0,0,0
5,1054311047,)and 8mo in 2009,and 8mo in 2009,0,0,0,0,0
6,1054668034,")HTN, hypercholesterolemia, DM, sleep apnea,, nephrolithiasis. chronic renal impairment, DVT since July 1977 on enoxaparin.",htn hypercholesterolemia dm sleep apnea nephrolithiasis chronic renal impairment dvt since jul 1977 on enoxaparin,0,0,0,0,0
7,1082469285,"Septeber, 10, 70 CPT Code: 90792: With medical services",sep 10 70 cpt code 90792 with medical services,0,0,0,0,0
8,1125769793,"Since 10/2014: Fatigued, more forgetful, impaired dexterity on her left hand. MRI reveals an approximately 4.2cm x 3.3cm x 2.5cm right parietal enhancing mass with surrounding edema",since 10/2014 fatigued more forgetful impaired dexterity on her left hand mri reveals an approximately 04 2cm x 03 3cm x 02 5cm right parietal enhancing mass with surrounding edema,0,0,0,0,0
9,1148116416,24 yo right handed woman with history of large right frontal mass s/p resection 11/3/1985 who had recent urgent R cranial wound revision and placement of L EVD for declining vision and increased drainage from craniotomy incision site and possible infection. She has a hx of secondary mania related to psychosis and manipulation of her right frontal lobe.,24 yo right handed woman with history of large right frontal mass s/p resection 11/03/1985 who had recent urgent r cranial wound revision and placement of l evd for declining vision and increased drainage from craniotomy incision site and possible infection she has a hx of secondary mania related to psychosis and manipulation of her right frontal lobe,0,0,0,0,0


## Helper Functions

In [10]:
def info(n=None, unmatched=True, head=5):
    "Helper function to display results of a match and some rows not matched so far."
    
    n = df.Match.max() if n is None else n
    for nn in ([n,0] if n!=0 and unmatched else [n]):
        display(Markdown("**Numer of rows with iter=%s: %s**" % (nn,len(df[df.Iter==nn]))))
        display(df.loc[df.Iter==nn,["Code","Data","Iter","Match", "Day","Month","Year"]].head(head))
info()

**Numer of rows with iter=0: 715**

,Code,Data,Iter,Match,Day,Month,Year
0,1012720972,12 nov 16 bad reaction to spicek2 synthetic mj admitted to crete manor mcalester,0,0,0,0,0
1,1039370009,apr 05 97 made a phone call to mom and mom commented that he was talking very fast hard to interrupt but was in super happy spirits so didn't make a big deal of it,0,0,0,0,0
2,1039574613,a pleasant 28 yo woman with no formal psychiatric history and with a h/o scca of the right tongue s/p partial glossectomy and neck dissection in 08/1974 referred to psycho oncology for assistance with adjustment issues following recovery the patient does not meet criteria for a major mood or anxiety disorder she is not at imminent risk of harm to self or others she would benefit from psychotherapy to help her integrate her experience of cancer and the break up of her engagement and to think through how to continue to create a life for herself moving forward,0,0,0,0,0
3,1039963589,oct 07 01 [report_end],0,0,0,0,0
4,1048901075,jul 04 01 primary care doctor,0,0,0,0,0


----
## Sequence of Regex's 

Feel free to change what I am doing here. The structure is simple, but adaptable:
 * Construct the regex
     * Pick a row and develop/test against it
 * Apply regex 
     * Set counter __iter__
     * apply to unmatched rows in __df__ to create a second dataframe, __df2__, with columns __Day__,__Month__,__Year__
     * Drop unmatch rows --- identified by NaN.
 * Test matched rows (I'm not giving you this, and you might not need it anyway, depending on your regex)
     * It is possible that the regex matched something that was not a valid date -- you want to remove these rows from __df2__. 
 * Update matched rows in __df__.
     * Save appropriate entry in __Day__, __Month__, __Year__, and __iter__.
     
---
---

### 1 - mm/dd/yyyy

In [11]:
tmp = df.loc[df.Code==2033924723,"Data"]
display(tmp)

# for month, first number can either start with 0 with range from 1-9 OR start with 1 with range from 1-2
regex = r"(?P<Month>0[1-9]|1[1-2]{1,2})/(?P<Day>[\d]{1,2})/(?P<Year>[\d]{2,4})"
tmp.str.extract(regex)

81    11/14/83 audit c score current
Name: Data, dtype: object

,Month,Day,Year
81,11,14,83


In [12]:
iter = 1

df2 = df[df.Iter==0].Data.str.extract(regex)
df2.dropna(inplace=True)
display(Markdown(" * Number of row matched = %s" % df2.shape[0]))
display(df2.head())

 * Number of row matched = 109

,Month,Day,Year
9,11,03,1985
10,04,19,91
14,07,29,1994
16,06,10,72
18,06,18,85


In [13]:
# save result
df.loc[(df.Iter==0) & (df2.Day.notnull()), "Day"] = df2["Day"]
df.loc[(df.Iter==0) & (df2.Month.notnull()), "Month"] = df2["Month"]
df.loc[(df.Iter==0) & (df2.Year.notnull()), "Year"] = df2["Year"]
df.loc[(df.Iter==0) & (df2.Year.notnull()),"Iter"] = iter

info(iter)

**Numer of rows with iter=1: 109**

,Code,Data,Iter,Match,Day,Month,Year
9,1148116416,24 yo right handed woman with history of large right frontal mass s/p resection 11/03/1985 who had recent urgent r cranial wound revision and placement of l evd for declining vision and increased drainage from craniotomy incision site and possible infection she has a hx of secondary mania related to psychosis and manipulation of her right frontal lobe,1,0,03,11,1985
10,1157934136,04/19/91 communication with referring physician? not done,1,0,19,04,91
14,1191233809,07/29/1994 cpt code 99203,1,0,29,07,1994
16,1218956332,06/10/72 sos 10 total score,1,0,10,06,72
18,1220889324,06/18/85 primary care doctor,1,0,18,06,85


**Numer of rows with iter=0: 606**

,Code,Data,Iter,Match,Day,Month,Year
0,1012720972,12 nov 16 bad reaction to spicek2 synthetic mj admitted to crete manor mcalester,0,0,0,0,0
1,1039370009,apr 05 97 made a phone call to mom and mom commented that he was talking very fast hard to interrupt but was in super happy spirits so didn't make a big deal of it,0,0,0,0,0
2,1039574613,a pleasant 28 yo woman with no formal psychiatric history and with a h/o scca of the right tongue s/p partial glossectomy and neck dissection in 08/1974 referred to psycho oncology for assistance with adjustment issues following recovery the patient does not meet criteria for a major mood or anxiety disorder she is not at imminent risk of harm to self or others she would benefit from psychotherapy to help her integrate her experience of cancer and the break up of her engagement and to think through how to continue to create a life for herself moving forward,0,0,0,0,0
3,1039963589,oct 07 01 [report_end],0,0,0,0,0
4,1048901075,jul 04 01 primary care doctor,0,0,0,0,0


### 2 - mm/yyyy

In [14]:
tmp = df.loc[df.Code==1039574613,"Data"]
display(tmp)

# for month, first number can either start with 0 with range from 1-9 OR start with 1 with range from 1-2
regex = r"(?P<Month>0[1-9]|1[1-2]{1,2})/(?P<Year>[\d]{2,4})"
tmp.str.extract(regex)

2    a pleasant 28 yo woman with no formal psychiatric history and with a h/o scca of the right tongue s/p partial glossectomy and neck dissection in 08/1974 referred to psycho oncology for assistance with adjustment issues following recovery the patient does not meet criteria for a major mood or anxiety disorder she is not at imminent risk of harm to self or others she would benefit from psychotherapy to help her integrate her experience of cancer and the break up of her engagement and to think through how to continue to create a life for herself moving forward 
Name: Data, dtype: object

,Month,Year
2,08,1974


In [15]:
iter = 2

df2 = df[df.Iter==0].Data.str.extract(regex)
df2.dropna(inplace=True)
display(Markdown(" * Number of row matched = %s" % df2.shape[0]))
display(df2.head())

 * Number of row matched = 98

,Month,Year
2,08,1974
11,03,2003
13,03,1990
35,02,96
44,05,2006


In [16]:
# save result
df.loc[(df.Iter==0) & (df2.Month.notnull()), "Month"] = df2["Month"]
df.loc[(df.Iter==0) & (df2.Year.notnull()), "Year"] = df2["Year"]
df.loc[(df.Iter==0) & (df2.Year.notnull()),"Iter"] = iter

info(iter)

**Numer of rows with iter=2: 98**

,Code,Data,Iter,Match,Day,Month,Year
2,1039574613,a pleasant 28 yo woman with no formal psychiatric history and with a h/o scca of the right tongue s/p partial glossectomy and neck dissection in 08/1974 referred to psycho oncology for assistance with adjustment issues following recovery the patient does not meet criteria for a major mood or anxiety disorder she is not at imminent risk of harm to self or others she would benefit from psychotherapy to help her integrate her experience of cancer and the break up of her engagement and to think through how to continue to create a life for herself moving forward,2,0,0,08,1974
11,1162722894,s6 past psychiatric hospitalizations starting at age 16 last 03/2003 for sib/si wwl x 2 getwell hospital lincoln hospital cox north lotus clinichx of outpatient treatment yes,2,0,0,03,2003
13,1174525826,knotice that in 03/1990 sustained a bizarre injury he was in colorado city at the time he was driving his car and he says he had recently ran out of saphris which is an antipsychotic he was taking he says he does not recall all the events but believes he stepped out of his vehicle and then walked off of a bridge sustaining a seven story fall he was found unconscious he was taken to and treated at norfolk health center in colorado city where he underwent open reduction internal fixation of the right humerus as well as the left femur was in icu for a week multiple fx he subsequently recovered from his injuries in the state of south carolina,2,0,0,03,1990
35,1445984744,10/02/96 age,2,0,0,02,96
44,1583575271,a endometriosis dx on laparoscopy 05/2006,2,0,0,05,2006


**Numer of rows with iter=0: 508**

,Code,Data,Iter,Match,Day,Month,Year
0,1012720972,12 nov 16 bad reaction to spicek2 synthetic mj admitted to crete manor mcalester,0,0,0,0,0
1,1039370009,apr 05 97 made a phone call to mom and mom commented that he was talking very fast hard to interrupt but was in super happy spirits so didn't make a big deal of it,0,0,0,0,0
3,1039963589,oct 07 01 [report_end],0,0,0,0,0
4,1048901075,jul 04 01 primary care doctor,0,0,0,0,0
5,1054311047,and 8mo in 2009,0,0,0,0,0


### 3 - 'month' dd/yyyy

In [17]:
tmp = df.loc[df.Code==7310415235,"Data"]
display(tmp)

regex = r"(?P<Month>jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)(?P<Day>\s+\d{1,2})(?P<Year>\s+\d{2,4})"
tmp.str.extract(regex)

510    jun 12 09 sos 10 total score
Name: Data, dtype: object

,Month,Day,Year
510,jun,12,09


In [18]:
iter = 3

df2 = df[df.Iter==0].Data.str.extract(regex)
df2.dropna(inplace=True)
display(Markdown(" * Number of row matched = %s" % df2.shape[0]))
display(df2.head())

 * Number of row matched = 159

,Month,Day,Year
1,apr,05,97
3,oct,07,01
4,jul,04,01
7,sep,10,70
26,jun,28,72


In [19]:
df.loc[(df.Iter==0) & (df2.Day.notnull()), "Day"] = df2["Day"]
df.loc[(df.Iter==0) & (df2.Month.notnull()), "Month"] = df2["Month"]
df.loc[(df.Iter==0) & (df2.Year.notnull()), "Year"] = df2["Year"]
df.loc[(df.Iter==0) & (df2.Year.notnull()),"Iter"] = iter

info(iter)

**Numer of rows with iter=3: 159**

,Code,Data,Iter,Match,Day,Month,Year
1,1039370009,apr 05 97 made a phone call to mom and mom commented that he was talking very fast hard to interrupt but was in super happy spirits so didn't make a big deal of it,3,0,05,apr,97
3,1039963589,oct 07 01 [report_end],3,0,07,oct,01
4,1048901075,jul 04 01 primary care doctor,3,0,04,jul,01
7,1082469285,sep 10 70 cpt code 90792 with medical services,3,0,10,sep,70
26,1339311197,jun 28 72 total time of visit in minutes,3,0,28,jun,72


**Numer of rows with iter=0: 349**

,Code,Data,Iter,Match,Day,Month,Year
0,1012720972,12 nov 16 bad reaction to spicek2 synthetic mj admitted to crete manor mcalester,0,0,0,0,0
5,1054311047,and 8mo in 2009,0,0,0,0,0
6,1054668034,htn hypercholesterolemia dm sleep apnea nephrolithiasis chronic renal impairment dvt since jul 1977 on enoxaparin,0,0,0,0,0
8,1125769793,since 10/2014 fatigued more forgetful impaired dexterity on her left hand mri reveals an approximately 04 2cm x 03 3cm x 02 5cm right parietal enhancing mass with surrounding edema,0,0,0,0,0
12,1164148605,stwin boys born 15 sep 91 gambling behavior no,0,0,0,0,0


### 4 - dd 'month' yyyy

In [20]:
tmp = df.loc[df.Code==1215203598,"Data"]
display(tmp)

regex = r"(?P<Day>\d{1,2})\s+(?P<Month>jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)\s+(?P<Year>[\d]{2,4})"
tmp.str.extract(regex)

15    14 jan 1981 sos 10 total score
Name: Data, dtype: object

,Day,Month,Year
15,14,jan,1981


In [21]:
iter = 4

df2 = df[df.Iter==0].Data.str.extract(regex)
df2.dropna(inplace=True)
display(Markdown(" * Number of row matched = %s" % df2.shape[0]))
display(df2.head())

 * Number of row matched = 188

,Day,Month,Year
0,12,nov,16
12,15,sep,91
15,14,jan,1981
17,01,sep,91
21,28,sep,93


In [22]:
df.loc[(df.Iter==0) & (df2.Day.notnull()), "Day"] = df2["Day"]
df.loc[(df.Iter==0) & (df2.Month.notnull()), "Month"] = df2["Month"]
df.loc[(df.Iter==0) & (df2.Year.notnull()), "Year"] = df2["Year"]
df.loc[(df.Iter==0) & (df2.Year.notnull()),"Iter"] = iter

info(iter)

**Numer of rows with iter=4: 188**

,Code,Data,Iter,Match,Day,Month,Year
0,1012720972,12 nov 16 bad reaction to spicek2 synthetic mj admitted to crete manor mcalester,4,0,12,nov,16
12,1164148605,stwin boys born 15 sep 91 gambling behavior no,4,0,15,sep,91
15,1215203598,14 jan 1981 sos 10 total score,4,0,14,jan,1981
17,1219337955,01 sep 91 communication with referring physician? not done,4,0,01,sep,91
21,1256289479,pt diagnosed in 28 sep 93 after he presented with 02 month history of headaches and gait instability mri demonstrated 04 cm l cereballar mass in the paravermian region he was admitted to prm and underwent resection complicated by post op delirium post op sequelas include left palatal myoclonus and ataxia on the left upper and lower extremities which has progressively improved pt has not had any evidence of tumor recurrence,4,0,28,sep,93


**Numer of rows with iter=0: 161**

,Code,Data,Iter,Match,Day,Month,Year
5,1054311047,and 8mo in 2009,0,0,0,0,0
6,1054668034,htn hypercholesterolemia dm sleep apnea nephrolithiasis chronic renal impairment dvt since jul 1977 on enoxaparin,0,0,0,0,0
8,1125769793,since 10/2014 fatigued more forgetful impaired dexterity on her left hand mri reveals an approximately 04 2cm x 03 3cm x 02 5cm right parietal enhancing mass with surrounding edema,0,0,0,0,0
22,1271563547,n abilify added to lexapro wellbutrin in jan 2007,0,0,0,0,0
24,1304079161,pt joined army reserves in 2001 and has 03 years left in this commitment mental status exam was the exam performed? if not indicate reason yes,0,0,0,0,0


### 5 - 'month' yyyy 

In [23]:
tmp = df.loc[df.Code==1054668034,"Data"]
display(tmp)

regex = r"(?P<Month>jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)\s+(?P<Year>\d{2,4})"
tmp.str.extract(regex)

6    htn hypercholesterolemia dm sleep apnea nephrolithiasis chronic renal impairment dvt since jul 1977 on enoxaparin
Name: Data, dtype: object

,Month,Year
6,jul,1977


In [24]:
iter = 5

df2 = df[df.Iter==0].Data.str.extract(regex)
df2.dropna(inplace=True)
display(Markdown(" * Number of row matched = %s" % df2.shape[0]))
df2.head()

 * Number of row matched = 97

,Month,Year
6,jul,1977
22,jan,2007
29,sep,1981
36,sep,1983
41,sep,1985


In [25]:
df.loc[(df.Iter==0) & (df2.Month.notnull()), "Month"] = df2["Month"]
df.loc[(df.Iter==0) & (df2.Year.notnull()), "Year"] = df2["Year"]
df.loc[(df.Iter==0) & (df2.Year.notnull()),"Iter"] = iter

info(iter)

**Numer of rows with iter=5: 97**

,Code,Data,Iter,Match,Day,Month,Year
6,1054668034,htn hypercholesterolemia dm sleep apnea nephrolithiasis chronic renal impairment dvt since jul 1977 on enoxaparin,5,0,0,jul,1977
22,1271563547,n abilify added to lexapro wellbutrin in jan 2007,5,0,0,jan,2007
29,1348968961,sgoiter diagnosed in sep 1981 pt feels thyroid problems are related to h/o lithium use,5,0,0,sep,1981
36,1451016026,ssep 1983 gsw to face l tmj region ? gang related with l cn vii injury and ? tbi requiring plastic surgical reconstructionactivities of daily living adl bathing independent,5,0,0,sep,1983
41,1524444733,s 20 yo m carries dx of bpad presents for psychopharm consult moved to independence area for school as of sep 1985,5,0,0,sep,1985


**Numer of rows with iter=0: 64**

,Code,Data,Iter,Match,Day,Month,Year
5,1054311047,and 8mo in 2009,0,0,0,0,0
8,1125769793,since 10/2014 fatigued more forgetful impaired dexterity on her left hand mri reveals an approximately 04 2cm x 03 3cm x 02 5cm right parietal enhancing mass with surrounding edema,0,0,0,0,0
24,1304079161,pt joined army reserves in 2001 and has 03 years left in this commitment mental status exam was the exam performed? if not indicate reason yes,0,0,0,0,0
32,1375836275,01 ex smoker quit 2012,0,0,0,0,0
34,1388842679,shx of tbi 1975 iso mva medical history,0,0,0,0,0


### 6 - mm dd yyyy

In [26]:
tmp = df.loc[df.Code==3173136043,"Data"]
display(tmp)

regex = r"(?P<Month>0[1-9]|1[1-2]{1,2})\s+(?P<Day>[\d]{1,2})\s+(?P<Year>[\d]{2,4})"
tmp.str.extract(regex)

176    07 29 75 cpt code 90801 psychiatric diagnosis interview
Name: Data, dtype: object

,Month,Day,Year
176,07,29,75


In [27]:
iter = 6

df2 = df[df.Iter==0].Data.str.extract(regex)
df2.dropna(inplace=True)
display(Markdown(" * Number of row matched = %s" % df2.shape[0]))
df2.head()

 * Number of row matched = 4

,Month,Day,Year
176,07,29,75
308,04,13,82
364,01,14,81
699,04,13,89


In [28]:
df.loc[(df.Iter==0) & (df2.Day.notnull()), "Day"] = df2["Day"]
df.loc[(df.Iter==0) & (df2.Month.notnull()), "Month"] = df2["Month"]
df.loc[(df.Iter==0) & (df2.Year.notnull()), "Year"] = df2["Year"]
df.loc[(df.Iter==0) & (df2.Year.notnull()),"Iter"] = iter

info(iter)

**Numer of rows with iter=6: 4**

,Code,Data,Iter,Match,Day,Month,Year
176,3173136043,07 29 75 cpt code 90801 psychiatric diagnosis interview,6,0,29,07,75
308,4895905688,04 13 82 other child mental health outcomes scales used,6,0,13,04,82
364,5561299129,01 14 81 communication with referring physician? done,6,0,14,01,81
699,9752673858,04 13 89 communication with referring physician? not done,6,0,13,04,89


**Numer of rows with iter=0: 60**

,Code,Data,Iter,Match,Day,Month,Year
5,1054311047,and 8mo in 2009,0,0,0,0,0
8,1125769793,since 10/2014 fatigued more forgetful impaired dexterity on her left hand mri reveals an approximately 04 2cm x 03 3cm x 02 5cm right parietal enhancing mass with surrounding edema,0,0,0,0,0
24,1304079161,pt joined army reserves in 2001 and has 03 years left in this commitment mental status exam was the exam performed? if not indicate reason yes,0,0,0,0,0
32,1375836275,01 ex smoker quit 2012,0,0,0,0,0
34,1388842679,shx of tbi 1975 iso mva medical history,0,0,0,0,0


### 7 - yyyy

In [29]:
tmp = df.loc[df.Code==1388842679,"Data"]
display(tmp)

regex = r"(?P<Year>\d{4})"
tmp.str.extract(regex)

34    shx of tbi 1975 iso mva medical history
Name: Data, dtype: object

,Year
34,1975


In [30]:
iter = 7

df2 = df[df.Iter==0].Data.str.extract(regex)
df2.dropna(inplace=True)
display(Markdown(" * Number of row matched = %s" % df2.shape[0]))
df2.head()

 * Number of row matched = 56

,Year
5,2009
8,2014
24,2001
32,2012
34,1975


In [31]:
df.loc[(df.Iter==0) & (df2.Year.notnull()), "Year"] = df2["Year"]
df.loc[(df.Iter==0) & (df2.Year.notnull()),"Iter"] = iter

info(iter)

**Numer of rows with iter=7: 56**

,Code,Data,Iter,Match,Day,Month,Year
5,1054311047,and 8mo in 2009,7,0,0,0,2009
8,1125769793,since 10/2014 fatigued more forgetful impaired dexterity on her left hand mri reveals an approximately 04 2cm x 03 3cm x 02 5cm right parietal enhancing mass with surrounding edema,7,0,0,0,2014
24,1304079161,pt joined army reserves in 2001 and has 03 years left in this commitment mental status exam was the exam performed? if not indicate reason yes,7,0,0,0,2001
32,1375836275,01 ex smoker quit 2012,7,0,0,0,2012
34,1388842679,shx of tbi 1975 iso mva medical history,7,0,0,0,1975


**Numer of rows with iter=0: 4**

,Code,Data,Iter,Match,Day,Month,Year
413,6219965646,npt denied use to me but endorsed use on 10/16/82 as part of bvh initial visit summary supplement by nicholas benjaminopiates yes,0,0,0,0,0
415,6223952005,10/24/86 communication with referring physician? done,0,0,0,0,0
418,6280461285,210/21/79 sos 10 total score,0,0,0,0,0
674,9513981482,10/29/91 communication with referring physician? done,0,0,0,0,0


In [32]:
## need to refactor regex rules with years, making sure it's EITHER 2 digits or 4

## need a rule that checks if the number is greter than 2019. If it is, it's likely a dd and mm missing a space, or mm and yy

## Check Count of Extracted Dates

In [33]:
mined_dates = df[(df.Iter!=0)]
print ("{0} of {1} dates have been mined".format(len(mined_dates), len(df)))

test = df[(df.Iter==7)]
test.head(50)

711 of 715 dates have been mined


,Code,Raw,Data,Iter,Match,Day,Month,Year
5,1054311047,)and 8mo in 2009,and 8mo in 2009,7,0,0,0,2009
8,1125769793,"Since 10/2014: Fatigued, more forgetful, impaired dexterity on her left hand. MRI reveals an approximately 4.2cm x 3.3cm x 2.5cm right parietal enhancing mass with surrounding edema",since 10/2014 fatigued more forgetful impaired dexterity on her left hand mri reveals an approximately 04 2cm x 03 3cm x 02 5cm right parietal enhancing mass with surrounding edema,7,0,0,0,2014
24,1304079161,"Pt joined Army reserves in 2001 and has 3 years left in this commitment.-Mental Status Exam Was the exam performed? (If not, indicate reason): Yes",pt joined army reserves in 2001 and has 03 years left in this commitment mental status exam was the exam performed? if not indicate reason yes,7,0,0,0,2001
32,1375836275,1 Ex-smoker : quit 2012,01 ex smoker quit 2012,7,0,0,0,2012
34,1388842679,shx of TBI (1975) ISO MVA.Medical History:,shx of tbi 1975 iso mva medical history,7,0,0,0,1975
68,1844017771,10/1973 Hx of Brain Injury: Yes,10/1973 hx of brain injury yes,7,0,0,0,1973
70,1858445338,"1. Esophageal cancer, dx: 2013, on FOLFOX with oxaliplatin desensitization",01 esophageal cancer dx 2013 on folfox with oxaliplatin desensitization,7,0,0,0,2013
71,1867893017,8Complications from brain hemmorage in 2016 Axis IV: Economic problems; Occupational problems; Problems related to social environment,8complications from brain hemmorage in 2016 axis iv economic problems; occupational problems; problems related to social environment,7,0,0,0,2016
79,1998978011,y1974 (all),y1974 all,7,0,0,0,1974
87,2141667325,"sSince 1998. Prior medication trials (including efficacy, reasons discontinued):",ssince 1998 prior medication trials including efficacy reasons discontinued,7,0,0,0,1998


In [34]:
# required rules


## Exporting

In [35]:
df.to_csv("results.csv", index=False)
from zipfile import ZipFile
archive = "date_assignment.zip"
print(f"Creating archive: {archive}")
with ZipFile(archive,"w") as zip:
    for f in ["01-Model.ipynb", "results.csv"]:
        if os.path.isfile(f):
            print(f"\t{f} - OK")
            zip.write(f) 
        else:
            print(f"\t{f} - Missing. Check this!")

Creating archive: date_assignment.zip
	01-Model.ipynb - OK
	results.csv - OK
